# Practical Session - Final

Student(s):

Aubery Cléa
Shinawi Aymeric

## 1. Problem selection

Choose one of the two following problems (delete the other cell).
You can find extra information regarding both problems and their associated models in
the main course.

### Problem 1: Single-Machine Scheduling

A machine needs to produce a set L of n items. Each
item i has a manufacturing time pi, a release time ri (time at which its components are
available) and a deadline di (time at which the time should be manufactured). The
factory can only produce one item at a time, and manufacturing process cannot be
interrupted (non-preemptive machine), what is the schedule that minimize the
overdue deliveries?

**Variables:**

- $x_{ij}$ - Binary variable indicating if $i$ is manufactured before $j$.
- $s_{i}$ - Continuous variable indicating the starting time of task $i$.
- $y_{i}$ - Binary variable indicating if the item $i$ is overdue.

**Model:**

$$
    \begin{align}
      \text{min.} \quad & \sum_{j=1}^{n} y_{i}                 &                                           \\
      \text{s.t.} \quad & s_j \geq s_i + p_i - M^1_{ij} (1 - x_{ij}), & \forall i,j \in L,\ i \ne j \\
                        & s_i + p_i \leq d_i + M^2_{i} y_i,          & \forall i \in L             \\
                        & x_{ij} + x_{ji} = 1,                 & \forall i,j \in L,\ i \ne j \\
                        & x_{ij} \in \left\{0,~1\right\},      & \forall i,j \in L                         \\
                        & y_{i} \in \left\{0,~1\right\},       & \forall i \in L                           \\
                        & s_{i} \geq r_{i},                    & \forall i \in L
    \end{align}
$$

## 2. Preparation

### 2.1. Instance

Implement a class to hold the various data parameters required for the problem (e.g.,
manufacturing time or aircraft target landing time).
Your class can be a simple data holder with various fields or can contain extra methods
for later use.

In [115]:
def createItem(pi, ri, di):
    return [pi, ri, di]

class Instance:
    def __init__(self):
      self.items=[]

    def addItems(self, items):
      self.items.extend(items)

    def addItem(self, item):
      self.items.append(item)

    def getItems(self):
      return self.items

    def getNbItems(self):
      return len(self.items)

    def getItem(self, i):
      return self.items[i]

    def getPi(self, i):
      return self.items[i][1]

    def getRi(self, i):
      return self.items[i][1]

    def getDi(self, i):
      return self.items[i][1]

In [116]:
# (Test)

print(createItem(2, 5, 10))

instance = Instance()

instance.addItem(createItem(2, 5, 10))
instance.addItem(createItem(1, 7, 15))
print(instance.getItems())

instance.addItems([createItem(3, 5, 22), createItem(4, 1, 8)])
print(instance.getItems())

print(instance.getItem(1))

[2, 5, 10]
[[2, 5, 10], [1, 7, 15]]
[[2, 5, 10], [1, 7, 15], [3, 5, 22], [4, 1, 8]]
[1, 7, 15]


### 2.2. Solution

Implement a to hold a proper solution for the chosen problem.
Your class should not contain the values for all variables in your model but only the
relevant ones.

In [117]:
# need to stock  ?
#    xij - Binary variable indicating if i is manufactured before j.  --> not really, can be deducted
#    yi - Binary variable indicating if the item i is overdue. --> not really, can be deducted
#    si - Continuous variable indicating the starting time of task i.  --> yes
#         How do we know i / the length of s ?
#         We have instance but not initialized.

class Solution:
    instance: Instance

    def __init__(self):
      self.s=[]

    def setInstance(self, instance):
      self.instance = instance
      self.s=[-1 for i in range(instance.getNbItems())]

    def setS(self, S):
      if(len(S)==len(self.s)):
        self.s = S
      else:
        raise Exception("Incorrect length for Y!")

    def setSi(self, i,  si):
      self.s[i]=si

    def getS(self):
      return self.s

    def getSi(self, i):
      return self.s[i]

    def getInstance(self):
      return self.instance

Implement the function below to check if a solution is valid or not.

In [59]:

def check_solution(solution: Solution) -> bool:
    instance = solution.getInstance()
    res = True
    for index, item in enumerate(instance.getItems()):
      # item  : ri, pi, di :  release time ri, manufacturing time pi and a deadline di
      # verification : task is executed
      soli = solution.getSi(index)

      if(soli==-1):
        res = False
      # verification : task is started after release time
      elif(item[0]>soli):
        res = False

    return res

def check_solution_ontime(solution : Solution) -> bool :
    res = check_solution(solution)
    for index, item in enumerate(instance.getItems()):
      soli = solution.getSi(index)
      # delay (exec + process > deadline)    
      # possible : return pair of boolean : isValid, isLate
      if item[1] + soli > item[2]:
        res = False
    return res
    

In [60]:
# (Test)
sol = Solution()

sol.setInstance(instance)
print(sol.getInstance().getItems())
print(sol.getS(), "\n")

sol.setS([1, 8, 15, 7])

# items = [[2, 5, 10], [1, 7, 15], [3, 5, 22], [4, 1, 8]]
# false sol
print(sol.getS())
print(check_solution(sol))

# false sol = 2, 8, 15, 7
sol.setSi(0, -1)
print(sol.getS())
print(check_solution(sol))


# true sol = 2, 8, 15, 7
sol.setSi(0, 2)
print(sol.getS())
print(check_solution(sol))
print(check_solution_ontime(sol))

# true sol (late) = 2, 8, 15, 7
sol.setSi(2, 20)
print(sol.getS())
print(check_solution(sol))
print(check_solution_ontime(sol))


[[2, 5, 10], [1, 7, 15], [3, 5, 22], [4, 1, 8]]
[-1, -1, -1, -1] 

[1, 8, 15, 7]
False
[-1, 8, 15, 7]
False
[2, 8, 15, 7]
True
True
[2, 8, 20, 7]
True
False


### 2.3. Data Generation

Propose, explain and implement a method to generate instances of various sizes (e.g.,
number of items or number of aircraft).

In [107]:
import random

def generate_instance(size: int, rng: random.Random = random.Random(), maxDuration: int = 1) -> Instance:
    # Generation of the n items. [pi, ri, di]
    # pi : On peut choisir une durée max des tâches en paramètre (défaut : 15)
    # ri : On choisi de façon aléatoire entre 0 et (pi-1)*size
    # di : On choisi de façon aléatoire entre pi et pi*size
    items = [[] for i in range(size)]
    for i in range(size):
      #
      ri = int (rng.random()*size*maxDuration)
      pi = int (rng.random()*maxDuration+1)

      di = int(rng.random()*maxDuration*4) + pi +ri
      items[i] = [[pi, ri, di]]

    return items


In [108]:
# (Test)

print(generate_instance(5))

[[[1, 1, 5]], [[1, 1, 2]], [[1, 4, 8]], [[1, 1, 3]], [[1, 0, 3]]]


## 3. Model implementation

### 3.1. Model

Implement the function below that should create an appropriate `docplex` model
given an instance of the chosen problem.

Use appropriate values for the big-$M$ constants in the model.

### Problem 1: Single-Machine Scheduling

A machine needs to produce a set L of n items. Each
item i has a manufacturing time pi, a release time ri (time at which its components are
available) and a deadline di (time at which the time should be manufactured). The
factory can only produce one item at a time, and manufacturing process cannot be
interrupted (non-preemptive machine), what is the schedule that minimize the
overdue deliveries?

**Variables:**

- $x_{ij}$ - Binary variable indicating if $i$ is manufactured before $j$.
- $s_{i}$ - Continuous variable indicating the starting time of task $i$.
- $y_{i}$ - Binary variable indicating if the item $i$ is overdue.

**Model:**

$$
    \begin{align}
      \text{min.} \quad & \sum_{j=1}^{n} y_{i}                 &                                           \\
      \text{s.t.} \quad & s_j \geq s_i + p_i - M^1_{ij} (1 - x_{ij}), & \forall i,j \in L,\ i \ne j \\
                        & s_i + p_i \leq d_i + M^2_{i} y_i,          & \forall i \in L             \\
                        & x_{ij} + x_{ji} = 1,                 & \forall i,j \in L,\ i \ne j \\
                        & x_{ij} \in \left\{0,~1\right\},      & \forall i,j \in L                         \\
                        & y_{i} \in \left\{0,~1\right\},       & \forall i \in L                           \\
                        & s_{i} \geq r_{i},                    & \forall i \in L
    \end{align}
$$

In [120]:
from docplex.mp.model import Model

def model_for(instance: Instance) -> Model:
    model = Model("Single-Machine Scheduling")
    
    L = instance.getNbItems()
    
    # upper bound for y : temps max *2 + biggest realease time
    M = max([pi for pi, ri, di in instance.getItems()]) * L + max([ri for pi, ri, di in instance.getItems()])

    # Create si, yi
    s = model.integer_var_list(L, 0, M, name="s_")
    y = model.binary_var_list(L, name="y_")
    
    # Create xi
    x = []
    for i in range(L):
        x.append(model.binary_var_list(L, name=f"x{i}_"))
        
    # Objective contraint
    model.minimize(sum(y))
    
    # --- Constraints ---------
    # sj >= si + pi - M(1, ij)(1-xij)
    for i in range(L):
        for j in range(L):
            if i != j:
                model.add_constraint(s[j] >= s[i] + instance.getItem(i)[0] - M * (1 - x[i][j]))

    # si + pi <= di + M(2, i)(yi)
    for i in range(L):
        model.add_constraint_(s[i] + instance.getPi(i) <= instance.getDi(i) + M * y[i])
      
    for i in range(L):
        # xij + xji = 1
        for j in range(i):
            model.add_constraint_(x[i][j]+x[j][i]==1)          
        
        # si >= ri    
        model.add_constraint_(s[i]>=instance.getRi(i))
        
    # -------------------------
        
    return model
    
print(model_for(instance))


docplex.mp.Model['Single-Machine Scheduling']


In [121]:
mdl = model_for(instance)
mdl

docplex.mp.Model['Single-Machine Scheduling']

### 3.2. Resolution

Complete the function below to construct a proper `Solution` from the obtained
`docplex` solution.

In [122]:
def solve(instance: Instance) -> Solution | None:
    model = model_for(instance)

    # enable logging
    model.log_output = True

    # solve
    solution = model.solve()
    
    if not solution:
        return None

    # TODO: convert the docplex solution to an appropriate Solution
    print("---------------")
    print(instance.getItems())
    print(solution)

In [123]:
solve(instance)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 4.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 8 rows and 8 columns.
MIP Presolve modified 18 coefficients.
Aggregator did 6 substitutions.
Reduced MIP has 12 rows, 10 columns, and 36 nonzeros.
Reduced MIP has 6 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 12 rows, 10 columns, and 36 nonzeros.
Reduced MIP has 6 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.08 ticks)
Parallel b&c, 12 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
T

## 4. Evaluation

### 4.1. Simple evaluation

Evaluate the model and solution implemented above on various instances generated by
your method.

What size of instances can your model solve? Which parameters have the biggest impact
on the resolution time?

### 4.2. Relaxation

Evaluate the impact of having tighter big-$M$ values in your model to the appropriate
relaxation.

**Tips:**

- Adapt the `model_for` function (or create a new one) to easily change the big-$M$
  values.
- You can use the following code to solve the relaxation of your model:

```python
from docplex.mp.relax_linear import LinearRelaxer

lp = LinearRelaxer.make_relaxed_model(model)
solution_relaxed = lp.solve(log_output=True)
```

### 4.3. Visualization (Bonus)

Propose a method to visualize the instance and solution obtained by your model using
library such as `matplotlib`.

In [138]:

# corriger pour permettre que les taches soient pas dans le même ordre
def visualization(instance: Instance, solution:Solution):
    current = 0
    
    for index, si in enumerate(solution.getS()):
        #pi, ri, di
        pi = instance.getPi(index)
        wait_time = abs(current-si)
        print("-"*wait_time, end="")
        print((str)(index)*pi, end="")
        current = pi+si

In [139]:
print(instance.getItems())
sol = Solution()
sol.setInstance(instance)
sol.setS([2, 8, 20, 30])
visualization(instance, sol)
#--00000-1111111-----22222-----3

[[2, 5, 10], [1, 7, 15], [3, 5, 22], [4, 1, 8]]
--00000-1111111-----22222-----3